# Overview AI search doc: https://learn.microsoft.com/en-us/python/api/overview/azure/search-documents-readme?view=azure-python

In [1]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from dotenv import load_dotenv

load_dotenv()

# Get the service endpoint and API key from the environment
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
key = os.environ["AZURE_SEARCH_ADMIN_KEY"]

index_name = "hotels"
search_client = SearchClient(endpoint, index_name, AzureKeyCredential(key))
search_client

<SearchClient [endpoint='https://cogsearchbasiceastus.search.windows.net', index='hotels']>

## Create Index 

In [2]:
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.indexes.models import SearchFieldDataType, CorsOptions, ScoringProfile, SearchIndex, SimpleField, SearchableField, ComplexField
from typing import List

# what is simplefield, searchablefield, complexfield?
# SimpleField: A simple field is a field that holds a single value of a primitive data type. 
# For example, a field that contains a string, integer, or double value is a simple field.
# Simple fields are the most common type of field in an index.

# SearchableField: A searchable field is a field that is full-text searchable.
# Searchable fields are used to search for text in the field.
# For example, a field that contains a description of a product is a searchable field.

# ComplexField: A complex field is a field that contains other fields.
# Complex fields are used to represent structured data.
# For example, a field that contains an address is a complex field.
# Complex fields can contain simple fields, searchable fields, and other complex fields.

# what is scoring profile
# A scoring profile is a set of weights that are used to rank search results.
# A scoring profile is used to boost the relevance of certain fields in the search results.
# For example, a scoring profile can be used to boost the relevance of the title field in the search results.

# what is cors options
# CORS (Cross-Origin Resource Sharing) is a security feature that allows web applications to make requests to a different domain.
# CORS is used to prevent web applications from making requests to a different domain.

Index_client = SearchIndexClient(endpoint, AzureKeyCredential(key))

fields = [
    SimpleField(name="hotelId", type=SearchFieldDataType.String, key=True),
    SimpleField(name="baseRate", type=SearchFieldDataType.Double),
    SearchableField(name="description", type=SearchFieldDataType.String, collection=True),
    ComplexField(
        name="address",
        fields=[
            SimpleField(name="streetAddress", type=SearchFieldDataType.String),
            SimpleField(name="city", type=SearchFieldDataType.String),
        ],
        collection=True,
    ),
]
cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)
scoring_profiles: List[ScoringProfile] = []
index = SearchIndex(name=index_name, fields=fields, scoring_profiles=scoring_profiles, cors_options=cors_options)

result = Index_client.create_index(index)
result

## Upload Json Doc to index

In [3]:
DOCUMENT = [
    {
        "hotelId": "abc123",
        "baseRate": 245.75,
        "description": [
            "Luxurious suite with ocean view",
            "Spacious rooms for families",
            "Cozy cabins in the woods"
        ],
        "address": [
            {
                "streetAddress": "123 Main Street",
                "city": "Seaside"
            },
            {
                "streetAddress": "456 Elm Avenue",
                "city": "Mountainville"
            }
        ]
    },
    {
        "hotelId": "xyz789",
        "baseRate": 189.50,
        "description": [
            "Budget-friendly rooms for travelers",
            "Historic inn with charming decor"
        ],
        "address": [
            {
                "streetAddress": "789 Oak Lane",
                "city": "Riverside"
            },
            {
                "streetAddress": "101 Pine Street",
                "city": "Hillside"
            }
        ]
    },
    {
        "hotelId": "pqr456",
        "baseRate": 310.25,
        "description": [
            "Mountain retreat with stunning views",
            "Rustic log cabins"
        ],
        "address": [
            {
                "streetAddress": "789 Forest Road",
                "city": "Woodland"
            },
            {
                "streetAddress": "222 Summit Drive",
                "city": "Peakville"
            }
        ]
    },
    {
        "hotelId": "def789",
        "baseRate": 199.99,
        "description": [
            "Quaint boutique hotel in the heart of the city",
            "Art deco architecture"
        ],
        "address": [
            {
                "streetAddress": "555 Broadway Avenue",
                "city": "Metropolis"
            },
            {
                "streetAddress": "777 Gallery Street",
                "city": "Artsville"
            }
        ]
    }
]

result = search_client.upload_documents(documents=DOCUMENT)

print("Upload of new document succeeded: {}".format(result[0].succeeded))

Upload of new document succeeded: True


In [4]:
query = 'Mountain views'

results = search_client.search(search_text = query)

for result in results:
    print(f"Hotel ID: {result['hotelId']}")
    print(f"Base Rate: ${result['baseRate']:.2f}")
    print("Description:")
    for desc in result['description']:
        print(f"  - {desc}")
    print("Address:")
    for addr in result['address']:
        print(f"  - Street Address: {addr['streetAddress']}")
        print(f"    City: {addr['city']}")
    print()

Hotel ID: pqr456
Base Rate: $310.25
Description:
  - Mountain retreat with stunning views
  - Rustic log cabins
Address:
  - Street Address: 789 Forest Road
    City: Woodland
  - Street Address: 222 Summit Drive
    City: Peakville



In [5]:
query = 'Ocean views'

results = search_client.search(search_text = query)

for result in results:
    print(f"Hotel ID: {result['hotelId']}")
    print(f"Base Rate: ${result['baseRate']:.2f}")
    print("Description:")
    for desc in result['description']:
        print(f"  - {desc}")
    print("Address:")
    for addr in result['address']:
        print(f"  - Street Address: {addr['streetAddress']}")
        print(f"    City: {addr['city']}")
    print()

Hotel ID: abc123
Base Rate: $245.75
Description:
  - Luxurious suite with ocean view
  - Spacious rooms for families
  - Cozy cabins in the woods
Address:
  - Street Address: 123 Main Street
    City: Seaside
  - Street Address: 456 Elm Avenue
    City: Mountainville

Hotel ID: pqr456
Base Rate: $310.25
Description:
  - Mountain retreat with stunning views
  - Rustic log cabins
Address:
  - Street Address: 789 Forest Road
    City: Woodland
  - Street Address: 222 Summit Drive
    City: Peakville



In [6]:
# !pip install aiohttp

## Async APIs

In [18]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.aio import SearchClient

search_client = SearchClient(endpoint, index_name, AzureKeyCredential(key))

async def search_hotels():
    results = await search_client.search(search_text="Ocean")
    async for result in results:
        print(f"hotelId: {result['hotelId']} (baseRate {result['baseRate']:.2f})")

# Call the async function
await search_hotels()


hotelId: abc123 (baseRate 245.75)


## Logging 
This library uses the standard logging library for logging. Basic information about HTTP sessions (URLs, headers, etc.) is logged at INFO level.

In [23]:
import sys
import logging
from azure.core.credentials import AzureKeyCredential

# Create a logger for the 'azure' SDK
logger = logging.getLogger('azure')
logger.setLevel(logging.DEBUG)

# Configure a console output
handler = logging.StreamHandler(stream=sys.stdout)
logger.addHandler(handler)

# Initialize the SearchClient with the provided credentials
search_client = SearchClient(endpoint, index_name, AzureKeyCredential(key), logging_enable=True)

search_client.search(search_text="mountain", logging_enable=True)

<SearchClient [endpoint='https://cogsearchbasiceastus.search.windows.net', index='hotels']>

In [24]:
Index_client.delete_index(index_name)